# Udacity 1st course

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

from sklearn.metrics.pairwise import euclidean_distances

# Randomized design tests

## 1st task
Use the data in Customer Service AB Testing Data file to perform an analysis on the following experiment and answer the questions that follows:

A call center for a cable internet provider wants to see if implementing an automated answering system to collect some data from the caller while they are on hold will increase customer service scores vs the customer service representative collecting that data after the customer has waited on hold. Twenty percent of the callers for the week of the experiment were randomly assigned to the treatment group and the rest were assigned to the control group. At the end of the call they were asked to fill out a one question survey. The data for the 1000 callers who took the survey is in the provided file.
- Think about the description of the experiment. Do you have any concerns about the setup of the experiment?
- what is the p-value for the experiment?
- Given this p-value, what would you conclude from the experiment?

### Think about the description of the experiment. Do you have any concerns about the setup of the experiment?
it would be helpful to know the total number of callers during the week of the experiment, if it's much larger, the sample size of 1000 might not be representative enough.  
we don't know  information on the response rate of the survey  
we don't know variables that could potentially influence customer service scores were controlled or accounted for.

In [2]:
df = pd.read_csv("customerserviceabtestdata.csv")

In [3]:
df.sample(5)

,RecordID,Phone Number,AutomatedFlag,CS Rating
271,272,(334) 689-2745,0,4
328,329,(326) 244-1876,0,1
428,429,(460) 193-1745,1,3
941,942,(621) 112-7318,1,2
443,444,(409) 589-8078,0,4


Using an average value for any customers who called in more than once

In [4]:
df_mto = df.groupby(["Phone Number", "AutomatedFlag"]).agg({"CS Rating": "mean"}).sort_values(by="Phone Number")

In [5]:
df_mto.sample(5)

,,CS Rating
Phone Number,AutomatedFlag,
(310) 459-9594,1,1.0
(347) 467-4244,0,2.0
(272) 997-2144,0,2.0
(663) 377-8444,0,1.0
(364) 971-9832,0,4.0


In [6]:
df.shape, df_mto.shape

((1000, 4), (992, 1))

In [7]:
df_mto.reset_index(inplace=True)

In [8]:
treatment = df_mto[df_mto["AutomatedFlag"] == 1]
control = df_mto[df_mto["AutomatedFlag"] == 0]

In [9]:
treatment.shape, control.shape

((247, 3), (745, 3))

In [10]:
print("Variances:")
np.var(treatment["CS Rating"]), np.var(control["CS Rating"])

Variances:


(1.8985067776885394, 1.9685797937029919)

### what is the p-value for the experiment?

In [11]:
# Perform the two sample t-test with equal variances
stats.ttest_ind(a=treatment["CS Rating"], b=control["CS Rating"], equal_var=True)

Ttest_indResult(statistic=1.9026387203223047, pvalue=0.057377625665938924)

### Given this p-value, what would you conclude from the experiment?

The p-value is above .05, which suggests there isn’t enough evidence to say that the automated system affects the CS score. However, due to the low sample size, I would suggest to conduct the experiment again for a longer period of time to gather more data.

# Matched pair design tests

## Task 1
Before we dive into how to use Alteryx to pick controls, let's try to do it manually. This will help you understand the process at a deeper level.

Using the New Product Control Data file, find the best control unit to match to Store ID 111. Feel free to use Excel, Google Sheets, or Alteryx.

Use the following variables as control variables to match on:  
Category Sales  
Number of Products  
State
- What store(s) would make the best control unit for Store ID 111?

In [12]:
df = pd.read_csv("newproductcontroldata.csv")

In [13]:
df.sample(5)

,Store ID,City,State,Zip Code,Category Sales,Product Count,Size
6889,6890,Spring,TX,77379,6145.60,4,Medium
6014,6015,Sherwood,AR,72120,18090.02,6,Large
2814,2815,Manhattan,KS,66503,34983.59,6,Large
2980,2981,Oak Creek,WI,53154,4750.51,5,Medium
2315,2316,Baraboo,WI,53913,7864.48,4,Large


In [14]:
df.dtypes

Store ID            int64
City               object
State              object
Zip Code            int64
Category Sales    float64
Product Count       int64
Size               object
dtype: object

In [15]:
state = df[df["Store ID"] == 111]["State"].to_string(index=False)
cat_sales = float(df[df["Store ID"] == 111]["Category Sales"].to_string(index=False))
num_of_prod = int(df[df["Store ID"] == 111]["Product Count"].to_string(index=False))

In [16]:
cat_sales-100

9456.14

In [17]:
df[(df["State"] == state) & (df["Category Sales"].between(cat_sales-50,
                                                          cat_sales+50)) & (df["Product Count"].between(num_of_prod-1, num_of_prod+1))]

,Store ID,City,State,Zip Code,Category Sales,Product Count,Size
110,111,Irvine,CA,92618,9556.14,5,Large
3497,3498,Santa Maria,CA,93455,9532.24,6,Large
5034,5035,Roseville,CA,95661,9602.96,5,Large


## Task 2
Let’s work through matching control units to a set of treatment units. For the new product introduction experiment let’s say that your manager has assigned a set of 10 treatment stores in California where the Grape flavor of the product will be sold.

The data for all the stores is in the file: NewProductControlData.csv

The data for the treatment stores for the Grape test is: GrapeProductTreatmentStores.csv

**What are the control stores for each treatment store?**

Your result should be a dataset with two fields: treatment store and control store. There are 10 treatment stores, and you're matching two control stores to each, so there should be twenty rows in your dataset. See the next section for the results.

In [18]:
control_data = pd.read_csv('newproductcontroldata.csv')
treatment_data = pd.read_csv('grapeproducttreatmentstores.csv')

In [19]:
control_data.sample(5)

,Store ID,City,State,Zip Code,Category Sales,Product Count,Size
78,79,CHATTANOOGA,TN,37415,38064.56,6,Large
5711,5712,Saint Louis,MO,63129,410553.94,8,Small
5536,5537,New York,NY,10169,10770.03,6,Small
5236,5237,Cleveland,TN,37312,176.80,2,Small
509,510,Worcester,MA,1605,190.00,4,Large


In [20]:
treatment_data.sample(5)

,Store ID,City,State,Zip Code,Category Sales,Product Count,Size
6,5019,Napa,CA,94558,16744.96,5,Large
0,332,La Mesa,CA,91942,56792.96,5,Large
1,944,Palmdale,CA,93551,41.50,1,Large
8,5395,Los Angeles,CA,90079,5669.56,5,Large
7,5051,Simi Valley,CA,93065,1378.85,3,Large


In [21]:
matching_fields = ['Product Count', 'Category Sales']
weights = [100, 1]

matched_stores = pd.DataFrame(columns=['treatment_store', 'control_store'])

for _, treatment_store in treatment_data.iterrows():
    control_data_filtered = control_data[control_data['State']
                                         == treatment_store['State']].reset_index(drop=True)
    control_data_filtered['Product Count'] *= weights[0]
    treatment_store['Product Count'] *= weights[0]
    distances = euclidean_distances(treatment_store[matching_fields].values.reshape(1, -1),
                                    control_data_filtered[matching_fields].values)

    top_indices = distances.argsort()[0][1:3]

    control_stores = control_data.loc[top_indices, 'Store ID'].tolist()

    matched_stores = pd.concat([
        matched_stores,
        pd.DataFrame({'treatment_store': [treatment_store['Store ID']] * 2,
                      'control_store': control_stores})
    ], ignore_index=True)

matched_stores

,treatment_store,control_store
0,332,782
1,332,608
2,944,95
3,944,675
4,1181,360
5,1181,623
6,2696,891
7,2696,126
8,2717,720
9,2717,55


## Task 3 / Mini Project

In [22]:
sales = pd.read_excel("sales-raw.xlsx", sheet_name="sales")

In [23]:
sales.sample(5)

,Region,Store,Week,Week_Start,Week_End,Period,Sum_Gross_Sales
459,West,store_527,54,2014-02-13,2014-02-19,Comp,2369.41
752,West,store_574,50,2014-01-16,2014-01-22,Comp,1993.24
880,West,store_593,58,2014-03-13,2014-03-19,Comp,5844.84
1646,Midwest,store_776,64,2014-04-24,2014-04-30,Test,645.35
63,Midwest,store_027,53,2014-02-06,2014-02-12,Comp,1926.59


### Step 1
Calculate the sales growth. You should create a pivot table to calculate the average sales in the comparable period for each store.

Calculate growth by using the formula:

(current_week_sales - avg_sales_comp) / avg_sales_comp

for each week for each store.

In [24]:
# Calculate average gross sales for the entire 'Comp' period using groupby
avg_sales_comp = sales[sales['Period'] == 'Comp'].groupby('Store')['Sum_Gross_Sales'].mean()
avg_sales_comp = avg_sales_comp.to_dict()

In [25]:
sales['avg_sales_comp'] = sales["Store"].map(avg_sales_comp)
sales['avg_gross_sales_growth'] = ((sales['Sum_Gross_Sales'] - sales['avg_sales_comp']) / sales['avg_sales_comp'])

In [26]:
sales.sample(5)

,Region,Store,Week,Week_Start,Week_End,Period,Sum_Gross_Sales,avg_sales_comp,avg_gross_sales_growth
1557,Midwest,store_760,55,2014-02-20,2014-02-26,Comp,3121.02,1332.071,1.342983
923,West,store_596,61,2014-04-03,2014-04-09,Test,1228.93,1942.662,-0.367399
1363,Midwest,store_724,61,2014-04-03,2014-04-09,Test,336.16,1050.332,-0.679949
1607,Midwest,store_767,65,2014-05-01,2014-05-07,Test,432.25,1450.457,-0.701990
686,West,store_566,62,2014-04-10,2014-04-16,Test,3005.75,1734.479,0.732941


### Step 2
Create a pivot table and average the growth by period. There should be two columns that contain an average for the comparable period and the average growth for the test period.

In [27]:
# Create a pivot table with average growth by period
pivot_table = pd.pivot_table(
    sales, values='avg_gross_sales_growth', index='Store', columns='Period', aggfunc='mean')

In [28]:
pivot_table.reset_index(inplace=True)
pivot_table

Period,Store,Comp,Test
0,store_012,7.216450e-17,0.098795
1,store_015,2.220446e-17,-0.542737
2,store_024,-8.881784e-17,0.199918
3,store_027,4.317534e-17,-0.211632
4,store_029,2.220446e-17,-0.187795
...,...,...,...
84,store_788,-1.110223e-17,0.109331
85,store_796,-6.106227e-17,-0.376040
86,store_798,-1.706968e-16,0.257464
87,store_802,1.443290e-16,-0.258828


In [29]:
pivot_table.rename({"Comp": "avg_growth_comparative_period",
                   "Test": "avg_growth_test_period"}, inplace=True, axis=1)

In [30]:
pivot_table["growth_difference"] = pivot_table["avg_growth_test_period"] - \
    pivot_table["avg_growth_comparative_period"]

### Step 3
Subtract the average growth in the test period from the average growth in the comparable period for each store.

In [31]:
control_treatment = pd.read_excel(
    "sales-raw.xlsx", sheet_name="treatment_control_pairs")

In [32]:
control_treatment.sample(5)

,Controls,Treatments,Distance,Region
7,store_550,store_695,0.193658,Midwest
13,store_636,store_769,0.399842,Midwest
8,store_577,store_755,0.209997,Midwest
26,store_776,store_024,0.590671,Midwest
32,store_788,store_696,0.096976,Midwest


In [33]:
control_treatment = control_treatment.merge(
    pivot_table[["Store", "growth_difference"]], left_on="Controls", right_on="Store", how="left")
control_treatment.rename(
    {"growth_difference": "diff_gross_sales_control"}, inplace=True, axis=1)
control_treatment.drop("Store", inplace=True, axis=1)

control_treatment = control_treatment.merge(
    pivot_table[["Store", "growth_difference"]], left_on="Treatments", right_on="Store", how="left")
control_treatment.rename(
    {"growth_difference": "diff_gross_sales_treatment"}, inplace=True, axis=1)
control_treatment.drop("Store", inplace=True, axis=1)

### Step 4
Calculate Lift and expected sales impact
The last step involves calculating lift from the growth difference. Lift is defined as the percent growth between the growth difference of a treatment store and its corresponding control store.

Lift = (Growth_Diff_Treatment_Store - Growth_Diff_Control_Store) / (1 + Growth_Diff_Control_Store)

Note: The reason why we add a number 1 in the denominator is because we're calculating a percent growth off a growth difference number which is merely a percentage difference. We need a percent gain number (ex: 1.20 instead of .20) to have a proper denominator for the lift calculation.

Once we get the lift number for each treatment store and its corresponding control stores, we take the average sales from the comparable period and multiple the average sales to get an average sales impact.

In [34]:
control_treatment["lift"] = (control_treatment["diff_gross_sales_treatment"] - control_treatment["diff_gross_sales_control"]) / (1 + control_treatment["diff_gross_sales_control"])

filtered_sales = sales[sales["Period"] == "Comp"][["Store", "avg_sales_comp"]].groupby("Store").mean().reset_index()
control_treatment = control_treatment.merge(
    filtered_sales, left_on="Treatments", right_on="Store", how="left")
control_treatment.drop("Store", inplace=True, axis=1)

control_treatment["expected_sales_impact"] = control_treatment["lift"] * \
    control_treatment["avg_sales_comp"]

In [35]:
control_treatment.head()

,Controls,Treatments,Distance,Region,diff_gross_sales_control,diff_gross_sales_treatment,lift,avg_sales_comp,expected_sales_impact
0,store_029,store_767,0.555454,Midwest,-0.187795,-0.283111,-0.117355,1450.457,-170.218896
1,store_033,store_769,0.172738,Midwest,-0.503942,0.026066,1.068439,1241.438,1326.400195
2,store_150,store_724,2.165599,Midwest,-0.153513,0.088115,0.285448,1050.332,299.815494
3,store_251,store_755,0.191664,Midwest,-0.032286,0.038626,0.073278,2178.119,159.607781
4,store_478,store_601,0.994041,West,-0.013892,-0.299704,-0.289838,2044.152,-592.473907


### Step 4
Calculate the lift and expected sales impact for each row.

Average the lift and expected sales impact.

In [36]:
control_treatment["lift"].mean(), control_treatment["expected_sales_impact"].mean()

(-0.0155801311983967, -122.38087755327379)

In [37]:
print(np.var(control_treatment["diff_gross_sales_control"]), np.var(control_treatment["diff_gross_sales_treatment"]))

0.03472870222794122 0.04369866996248451


In [38]:
print(stats.ttest_ind(control_treatment["diff_gross_sales_control"],
      control_treatment["diff_gross_sales_treatment"], equal_var=True))

Ttest_indResult(statistic=1.5741204864558072, pvalue=0.12024142839870754)
